In [1]:
import json
import pandas as pd
import datetime

%matplotlib inline

In [2]:
with open('donnees.json', encoding='utf-8') as mon_fichier:
    dictionnaire_des_participants = json.load(mon_fichier)

# For python analysis

In [3]:
def temps_de_reponse(se):
    values = se.get_time.values[0][:-3]
    date1 = datetime.datetime(*values)
    values = se.post_time.values[0][:-3]
    date2 = datetime.datetime(*values)

    delta = (date2 - date1)
    return delta.seconds

In [4]:
def trial_to_bloc(se):
    if se.trial.values[0] <= 16:
        return 1
    if se.trial.values[0] <= 32:
        return 2
    if se.trial.values[0] <= 48:
        return 3
    if se.trial.values[0] <= 64:
        return 4
    raise Exception('Invalid trial')

In [5]:
dict_columns = ['BaseDeDonnees', 'CommentairesGeneraux', 'Consentement_form', 'CriteresInclusion', 'Demographique', 'ProfilJeuxVideo', 'SocioDemo',]

dfs = []

for par, data in enumerate(dictionnaire_des_participants.values()):
    dictionary = {key: data[key] for key in data.keys() if key in dict_columns}
    reform = {(outerKey, innerKey): values for outerKey, innerDict in dictionary.items() for innerKey, values in innerDict.items()}
    se = pd.Series(reform)
    
    for key, val in data['data'].items():
        val.update({'avatar': data['order'][int(key)-1]})
        
    df = pd.DataFrame.from_dict(data['data'], orient='index')
    df = df.reset_index().rename(columns={'index':'trial'})
    df = df.astype({'trial':int, 'avatar':int})
    df.columns = pd.MultiIndex.from_product([df.columns, ['']])

    for rows in se.items():
        if isinstance(rows[1], list):
            df[rows[0]] = ';'.join(rows[1])
        else:
            df[rows[0]] = rows[1]
    
    df['participant'] = par
    dfs.append(df)
    
df_all = pd.concat(dfs)
df_all['temps_de_reponse'] = df_all.apply(temps_de_reponse, axis=1)
df_all['male'] = df_all.apply(lambda se: True if se.Demographique.sexe =='Masculin' else False, axis=1)
df_all['bloc'] = df_all.apply(trial_to_bloc, axis=1)
df_all['age_trial'] = df_all.age > -1
df_all['realism_trial'] = df_all.realisme > -1
df_all['age_participant'] = (pd.datetime.now() - pd.to_datetime(df_all.Demographique.datenaissance)).apply(lambda x: x.days//365)

# remove invalide participant
invalid_participants = list(df_all.groupby('participant').realisme.count()[(df_all.groupby('participant').realisme.count() != 32)].index)
df_all = df_all[~df_all.participant.isin(invalid_participants)]


df_all.to_pickle('eevee-data.pkl')

In [6]:
lines = []
for participant in df_all.participant.unique():
    for avatar in df_all.avatar.unique():
        df_par = df_all[(df_all.participant == participant) & (df_all.avatar == avatar)]
        line = df_par.iloc[0].drop(['age', 'genre', 'realisme', 'temps_de_reponse', 'bloc', 'age_trial', 'realism_trial', 'get_time', 'post_time', 'repeat', 'groupe_ethnique', 'groupe_ethnique_autre'], level=0)
        
        line['age1'] = df_par[df_par.bloc==1].age.values[0]
        line['age2'] = df_par[df_par.bloc==3].age.values[0]
        
        line['genre1'] = df_par[df_par.bloc==1].genre.values[0]
        line['genre2'] = df_par[df_par.bloc==3].genre.values[0]
        
        line['groupe_ethnique1'] = df_par[df_par.bloc==1].groupe_ethnique.values[0]
        line['groupe_ethnique2'] = df_par[df_par.bloc==3].groupe_ethnique.values[0]
        
        line['groupe_ethnique_autre1'] = df_par[df_par.bloc==1].groupe_ethnique_autre.values[0]
        line['groupe_ethnique_autre2'] = df_par[df_par.bloc==3].groupe_ethnique_autre.values[0]
        
        line['realisme1'] = df_par[df_par.bloc==2].realisme.values[0]
        line['realisme2'] = df_par[df_par.bloc==4].realisme.values[0]
        
        lines.append(line)
        
df_spss = pd.DataFrame(lines)
df_spss = df_spss.sort_values(['participant', 'avatar']).drop('trial', axis=1, level=1)
df_spss.columns = list(map(str.strip, map(' '.join, list(df_spss.columns))))
df_spss = df_spss.drop('trial', axis=1)


In [7]:
writer = pd.ExcelWriter('Donnees_Spss.xlsx')
df_spss.to_excel(writer,'Sheet1')
writer.save()